**Table of contents**<a id='toc0_'></a>    
- [AR Marker Detection](#toc1_)    
- [Detecting AR Markers Using Pybullet](#toc2_)    
  - [Starting pybullet](#toc2_1_)    
  - [Initial Setup for pybullet](#toc2_2_)    
  - [Generating the Camera and the Box with AR Marker](#toc2_3_)    
  - [Changing the Light Source Position](#toc2_4_)    
  - [Defining the Function to Detect AR Markers](#toc2_5_)    
  - [Defining Camera Parameters](#toc2_6_)    
  - [Defining debugParameter](#toc2_7_)    
  - [Running AR Marker Detection](#toc2_8_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[AR Marker Detection](#toc0_)

In this notebook, we will explain how to obtain the position and orientation of a box with an AR marker attached for use in pybullet.

For a manual summarizing the functions available in pybullet, refer to [this link](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf).

# <a id='toc2_'></a>[Detecting AR Markers Using Pybullet](#toc0_)

Next, we will explain how to detect AR markers using pybullet.

## <a id='toc2_1_'></a>[Starting pybullet](#toc0_)


In [2]:
import pybullet
import pybullet_data
physics_client = pybullet.connect(pybullet.GUI) 

pybullet build time: Nov 28 2023 23:45:17


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa
GL_RENDERER=llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Mesa
Renderer = llvmpipe (LLVM 15.0.7, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


## <a id='toc2_2_'></a>[Initial Setup for pybullet](#toc0_)

We will perform initial setup for pybullet, such as creating the floor and setting up the camera.

In [3]:
pybullet.resetSimulation() # Reset the simulation space
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # Add paths to necessary data for pybullet
pybullet.setGravity(0.0, 0.0, -9.8) # Set gravity as on Earth
time_step = 1./240.
pybullet.setTimeStep(time_step) # Set the time elapsed per step

# Load the floor
plane_id = pybullet.loadURDF("plane.urdf")

# Set the camera position and other parameters in GUI mode
camera_distance = 1.0
camera_yaw = -150.0 # deg
camera_pitch = -20 # deg
camera_target_position = [0.0, 0.0, 0.1]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)


ven = Mesa
ven = Mesa


## <a id='toc2_3_'></a>[Generating the Camera and the Box with AR Marker](#toc0_)

Next, we will generate the box with the AR marker attached and the camera model to detect the AR marker.


In [ ]:
# Load the camera (the camera size is 0.1m x 0.1m x 0.1m, so move it 0.05m in the y-axis direction from the origin to align the camera front with the origin)
simple_camera_id = pybullet.loadURDF("../urdf/simple_camera.urdf", [0.0, 0.05, 0.5], pybullet.getQuaternionFromEuler([1.57, 0.0, 0.0]), useFixedBase=True)

# Load the box with the AR marker attached
# Set the position so that the distance from the front of the AR marker box to the camera front is 1m
ar_marker_box_id = pybullet.loadURDF("../urdf/ar_marker_box.urdf", [0.0, -1.07, 0.5], pybullet.getQuaternionFromEuler([0.0, 0.0, 0.0]), useFixedBase=True)

# Set the texture (specify the same one as in the urdf file)
texture_id = pybullet.loadTexture("../texture/ar_marker_box.png")
pybullet.changeVisualShape(ar_marker_box_id, -1, textureUniqueId=texture_id)

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: base_link
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: target_position_vertual_link


## <a id='toc2_4_'></a>[Changing the Light Source Position](#toc0_)

Change the position of the light source to make it easier to detect the AR marker.

In [5]:
# Change the light source position because the AR marker is hard to recognize with the default light source position
pybullet.configureDebugVisualizer(lightPosition=[0, 0, 100])

## <a id='toc2_5_'></a>[Defining the Function to Detect AR Markers](#toc0_)

Define a function to obtain the position and orientation of the AR marker from the acquired image information.

In [6]:
import cv2
import numpy as np

def detect_ar_marker_pose(marker_size, aruco_dict, parameters, rgb_img, camera_matrix, dist_coeffs):
    """
    Function to obtain the center position, depth, and orientation of the first detected AR marker
    
    Parameters
    ----------
    marker_size : float
        Length of one side of the AR marker (meters)
    aruco_dict : cv2.aruco.Dictionary
        Dictionary of AR markers
    parameters : cv2.aruco.DetectorParameters
        Parameters for AR marker detection
    rgb_img : numpy.ndarray
        Camera image (RGB)
    camera_matrix : numpy.ndarray
        Camera intrinsic parameter matrix
    dist_coeffs : numpy.ndarray
        Distortion coefficients

    Returns
    -------
    marker_pose : numpy.ndarray
        Position and orientation of the AR marker (x, y, z, roll, pitch, yaw)
    """

    # Convert the camera image to OpenCV format
    bgr_img = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2BGR)

    # Detect AR markers
    corners, ids, _ = cv2.aruco.detectMarkers(bgr_img, aruco_dict, parameters=parameters)

    # Return None if no markers are detected
    if ids is None or len(ids) == 0:
        return None

    # Get the position of the four corners of the first detected marker
    corner_position_2d = corners[0][0]

    # 3D coordinates of the four corners of the marker
    corner_position_3d = np.array([[-marker_size/2, -marker_size/2, 0],
                                    [marker_size/2, -marker_size/2, 0],
                                    [marker_size/2, marker_size/2, 0],
                                    [-marker_size/2, marker_size/2, 0]], dtype=np.float32)
    
    # Use solvePnP to calculate the distance from the camera to the AR marker
    _, rotation_vector, translation_vector = cv2.solvePnP(corner_position_3d, corner_position_2d, camera_matrix, dist_coeffs)

    # Get the center position and depth of the marker
    tvec = np.array(translation_vector)
    x = tvec[0][0]
    y = tvec[1][0]
    z = tvec[2][0]

    # Get the orientation of the marker
    rvec = np.array(rotation_vector)
    roll = rvec[0][0]
    pitch = rvec[1][0]
    yaw = rvec[2][0]

    # Return the position and orientation of the marker
    marker_pose = np.array([x, y, z, roll, pitch, yaw])

    return marker_pose


## <a id='toc2_6_'></a>[Defining Camera Parameters](#toc0_)

Define the parameters of the camera used to detect the AR marker.

In [7]:
# Camera settings
fov = 60
image_width = 640
image_height = 480
aspect = image_width / image_height
near = 0.05
far = 10
projection_matrix = pybullet.computeProjectionMatrixFOV(fov, aspect, near, far)

# Calculate the focal length in the y direction
fovRad = np.deg2rad(fov)
f = image_height / (2 * np.tan(fovRad / 2))

# Camera intrinsic parameters
camera_matrix = np.array([[f, 0, image_width/2],
                          [0, f, image_height/2],
                          [0, 0, 1]], dtype=np.float32)

# Distortion coefficients (assuming no distortion here)
dist_coeffs = np.array([0.0, 0.0, 0.0, 0.0, 0.0], dtype=np.float32)

# Specify the indices of the "camera link" and the "virtual link for capturing camera images"
CAMERA_LINK_IDX = 0
CAMERA_TARGET_LINK_IDX = 1

## <a id='toc2_7_'></a>[Defining debugParameter](#toc0_)

Define parameters to change the position and orientation of the box with the AR marker attached using sliders.

In [8]:
pybullet.addUserDebugParameter("camera x", -4, 4, 0.0)
pybullet.addUserDebugParameter("camera y", -4, 4, -1.07) # Set the initial position so that the distance from the front of the AR marker box to the camera front is 1m
pybullet.addUserDebugParameter("camera z", -4, 8, 0.5)
pybullet.addUserDebugParameter("camera roll", -3.14, 3.14, 0.0)
pybullet.addUserDebugParameter("camera pitch", -3.14, 3.14, 0)
pybullet.addUserDebugParameter("camera yaw", -3.14, 3.14, 0)

5

## <a id='toc2_8_'></a>[Running AR Marker Detection](#toc0_)

When you run the code below, the position and orientation of the AR marker detected by the camera will be displayed on the Pybullet GUI screen.

Additionally, you can change the position and orientation of the "box with the AR marker attached" by moving the sliders on the right side of the GUI screen.

In [9]:
import time

# Size of the AR marker to be detected (specify the length of one side of the AR marker)
marker_size = 0.1

# Define the dictionary for the AR marker to be detected
aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
parameters = cv2.aruco.DetectorParameters_create()

while True:
    # Get the values set in the sliders
    camera_x = pybullet.readUserDebugParameter(0)
    camera_y = pybullet.readUserDebugParameter(1)
    camera_z = pybullet.readUserDebugParameter(2)
    camera_roll = pybullet.readUserDebugParameter(3)
    camera_pitch = pybullet.readUserDebugParameter(4)
    camera_yaw = pybullet.readUserDebugParameter(5)

    # Update the position and orientation of the box with the AR marker attached
    pybullet.resetBasePositionAndOrientation(ar_marker_box_id, [camera_x, camera_y, camera_z], pybullet.getQuaternionFromEuler([camera_roll, camera_pitch, camera_yaw]))

    # Get the position of the camera
    camera_link_pose = pybullet.getLinkState(simple_camera_id, CAMERA_LINK_IDX)[0] # Position of the camera link at the end of the arm
    camera_target_link_pose = pybullet.getLinkState(simple_camera_id, CAMERA_TARGET_LINK_IDX)[0] # Position of a virtual link set just in front of the camera link

    # Compute the view matrix of the camera
    view_matrix = pybullet.computeViewMatrix(cameraEyePosition=[camera_link_pose[0], camera_link_pose[1], camera_link_pose[2]], cameraTargetPosition=[camera_target_link_pose[0], camera_target_link_pose[1], camera_target_link_pose[2]], cameraUpVector=[0, 0, 1])

    # Get the camera image
    _, _, rgb_img, _, _ = pybullet.getCameraImage(
        width=image_width,
        height=image_height,
        viewMatrix=view_matrix,
        projectionMatrix=projection_matrix,
        renderer=pybullet.ER_BULLET_HARDWARE_OPENGL
    )

    # Get the position and orientation of the AR marker
    marker_pose = detect_ar_marker_pose(marker_size, aruco_dict, parameters, rgb_img, camera_matrix, dist_coeffs)
    
    # If detection fails, continue to the next loop
    if marker_pose is None:
        continue

    # Display the position and orientation of the AR marker
    x = marker_pose[0]
    y = marker_pose[2]
    z = -marker_pose[1]
    roll = -marker_pose[3]
    pitch = -marker_pose[5]
    yaw = -marker_pose[4]
    pybullet.addUserDebugText(f"marker pose ({x:.2f}, {y:.2f}, {z:.2f}, {np.rad2deg(roll):.2f}, {np.rad2deg(pitch):.2f}, {np.rad2deg(yaw):.2f})", [1.2, 0, 0], textColorRGB=[1, 0, 0], textSize=1.5, lifeTime=0.5)

    time.sleep(time_step)